In [1]:
import subprocess
import shlex
import re

import numpy as np

from scipy.stats import combine_pvalues
from IPython.display import display, Markdown

In [2]:
RANDOMNESS_PATTERN = re.compile(r"exceed this value (\d+\.\d+) percent")

CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

# ENT - pseudorandom number sequence test

In [3]:
def test_randomness_ent(
    mode, seeds=[42], limit='1GB',
):
    outputs = []
    
    for seed in seeds:
        generator = subprocess.Popen(shlex.split(
            f"cargo run --release -- --seed {seed} --limit {limit} {CC_PATTERN.sub('-', mode).lower()}"
        ), stdout=subprocess.PIPE)
        ent = subprocess.run(
            shlex.split("./ent"), stdin=generator.stdout,
            check=True, capture_output=True, text=True
        )

        pvalue = float(RANDOMNESS_PATTERN.search(ent.stdout).group(1)) / 100.0
        
        outputs.append((pvalue, ent.stdout))
    
    gof_statistic, gof_pvalue = combine_pvalues([pvalue for pvalue, output in outputs])

    if gof_pvalue <= 0.01 or gof_pvalue >= 0.99:
        display(Markdown(f"## <span style='color:purple'><u>{mode}</u></span>"))
    elif gof_pvalue <= 0.05 or gof_pvalue >= 0.95:
        display(Markdown(f"## <span style='color:red'><u>{mode}</u></span>"))
    elif gof_pvalue <= 0.1 or gof_pvalue >= 0.9:
        display(Markdown(f"## <span style='color:orange'>*{mode}*</span>"))
    else:
        display(Markdown(f"## <span style='color:green'>{mode}</span>"))
        
    most_suspect = None
    
    for (pvalue, output) in outputs:
        suspicion = 1 - min(pvalue, 1 - pvalue)
        
        if (most_suspect is None) or (suspicion > most_suspect[0]):
            most_suspect = (suspicion, output)
        
    display(Markdown("#### Fisher’s combined Chi-squared:"))
    display(Markdown(f"* p-value: {gof_pvalue}\n* statistic: {gof_statistic}"))

    display(Markdown("#### Test output (most suspect):"))
    display(Markdown(">" + most_suspect[1].replace('\n', '\n>')))

    display(Markdown("#### Parameters:"))
    display(Markdown(f"* seeds: {seeds}\n* limit: {limit}"))

In [4]:
for mode in ["Monolithic", "Independent", "IndependentSimulation", "IndependentSimulationNoDispersal"]:
    seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=10)
    
    test_randomness_ent(mode, seeds=seeds, limit='100MB')

## <span style='color:orange'>*Monolithic*</span>

#### Fisher’s combined Chi-squared:

* p-value: 0.9203830053780299
* statistic: 11.87284504453826

#### Test output (most suspect):

>Entropy = 7.999998 bits per byte.
>
>Optimum compression would reduce the size
>of this 100000000 byte file by 0 percent.
>
>Chi square distribution for 100000000 samples is 217.27, and randomly
>would exceed this value 95.83 percent of the times.
>
>Arithmetic mean value of data bytes is 127.5069 (127.5 = random).
>Monte Carlo value for Pi is 3.141757806 (error 0.01 percent).
>Serial correlation coefficient is 0.000141 (totally uncorrelated = 0.0).
>

...

* p-value: 0.15112128749977102
* statistic: 26.460976353035672

#### Test output (most suspect):

>Entropy = 7.999998 bits per byte.
>
>Optimum compression would reduce the size
>of this 100000000 byte file by 0 percent.
>
>Chi square distribution for 100000000 samples is 297.67, and randomly
>would exceed this value 3.42 percent of the times.
>
>Arithmetic mean value of data bytes is 127.4839 (127.5 = random).
>Monte Carlo value for Pi is 3.142323726 (error 0.02 percent).
>Serial correlation coefficient is -0.000005 (totally uncorrelated = 0.0).
>

#### Parameters:

* seeds: [17770599855847412276 15708050387024954144 15035841265197839949
  2523591470737598723  3735631641601737476 11099102083812552322
  5828206479837106746  8530539764271179718 11408569139797663507
 13707293129912994060]
* limit: 100MB